In [0]:
import math

import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.datasets import mnist
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Dropout, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

In [0]:
NUM_CLASS = 10

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data(path='mnist.npz')

In [0]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

In [43]:
print(len(x_train))
print(len(x_test))

60000
10000


In [44]:
x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = to_categorical(y_train, NUM_CLASS)
y_test = to_categorical(y_test, NUM_CLASS)

print("x_train.shape = {}, y_train.shape = {}".format(x_train.shape, y_train.shape))
print("x_test.shape = {}, y_test.shape = {}".format(x_test.shape, y_test.shape))

x_train.shape = (60000, 28, 28, 1), y_train.shape = (60000, 10)
x_test.shape = (10000, 28, 28, 1), y_test.shape = (10000, 10)


In [0]:
inputs = Input(shape=(28, 28, 1), name='input')

x = Conv2D(24, kernel_size=(6, 6), strides=1)(inputs)
x = BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = Activation('relu')(x)
x = Dropout(rate=0.25)(x)

x = Conv2D(48, kernel_size=(5, 5), strides=2)(x)
x = BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = Activation('relu')(x)
x = Dropout(rate=0.25)(x)

x = Conv2D(64, kernel_size=(4, 4), strides=2)(x)
x = BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = Activation('relu')(x)
x = Dropout(rate=0.25)(x)

x = Flatten()(x)
x = Dense(200)(x)
x = BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = Activation('relu')(x)
x = Dropout(rate=0.25)(x)

predications = Dense(NUM_CLASS, activation='softmax', name='output')(x)

model = Model(inputs=inputs, outputs=predications)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [46]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 23, 23, 24)        888       
_________________________________________________________________
batch_normalization_12 (Batc (None, 23, 23, 24)        72        
_________________________________________________________________
activation_12 (Activation)   (None, 23, 23, 24)        0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 23, 23, 24)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 10, 10, 48)        28848     
_________________________________________________________________
batch_normalization_13 (Batc (None, 10, 10, 48)        144 

In [47]:
lr_decay = lambda epoch: 0.0001 + 0.02 * math.pow(1.0 / math.e, epoch / 3.0)
decay_callback = LearningRateScheduler(lr_decay, verbose=1)

history = model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=1, 
                    validation_data=(x_test, y_test), callbacks=[decay_callback])

Train on 60000 samples, validate on 10000 samples

Epoch 00001: LearningRateScheduler reducing learning rate to 0.0201.
Epoch 1/10
60000/60000 [==============================] - 9s 145us/sample - loss: 0.1559 - acc: 0.9527 - val_loss: 0.0477 - val_acc: 0.9861

Epoch 00002: LearningRateScheduler reducing learning rate to 0.014430626211475785.
Epoch 2/10
60000/60000 [==============================] - 8s 133us/sample - loss: 0.0618 - acc: 0.9813 - val_loss: 0.0404 - val_acc: 0.9876

Epoch 00003: LearningRateScheduler reducing learning rate to 0.01036834238065184.
Epoch 3/10
60000/60000 [==============================] - 8s 132us/sample - loss: 0.0468 - acc: 0.9854 - val_loss: 0.0311 - val_acc: 0.9912

Epoch 00004: LearningRateScheduler reducing learning rate to 0.007457588823428847.
Epoch 4/10
60000/60000 [==============================] - 8s 133us/sample - loss: 0.0334 - acc: 0.9890 - val_loss: 0.0241 - val_acc: 0.9920

Epoch 00005: LearningRateScheduler reducing learning rate to 0.00537

In [50]:
model.input_shape

(None, 28, 28, 1)

In [0]:
model.save('model.h5')

In [49]:
converter = tf.lite.TFLiteConverter.from_keras_model_file('model.h5')
tflite_model = converter.convert()
open('mnist-model.tflite', 'wb').write(tflite_model)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 22 variables.
INFO:tensorflow:Converted 22 variables to const ops.


1146028